# Find SLiMs in interest regions in amyloid protein dataset
From bibliographical reference (1), we have a set of amyloid proteins with possible instances of short linear motifs, in this notebook, we aim to find if those regions match any known classes of motif in the ELM resource.

1.	Beg, A. Z. & Khan, A. U. Motifs and interface amino acid-mediated regulation of amyloid biogenesis in microbes to humans: potential targets for intervention. Biophys. Rev. 12, 1249–1256 (2020).

## Imports

In [1]:
import Bio.SeqIO
import pandas as pd
import re

## Loading protein data
We used alignments for each protein mentioned in the paper (1) to make the search with the patterns more thorough.
Loading the alignments of each protein, the region of interest and other relevant data into custom data structures for ease of use.

In [2]:
class Protein:
    def __init__(self,unaligned_path,aligned_path,query,myseq):
        self.unaligned_path = unaligned_path
        self.aligned_path = aligned_path
        self.query = query
        self.myseq = myseq
        unaligned_dict = {}
        for sequence in Bio.SeqIO.parse(self.unaligned_path, "fasta"):
            unaligned_dict[sequence.id] = sequence.seq
        self.unaligned_dict = unaligned_dict
        aligned_dict = {}
        for sequence in Bio.SeqIO.parse(self.aligned_path, "fasta"):
            aligned_dict[sequence.id] = sequence.seq
        self.aligned_dict = aligned_dict
    def __repr__(self):
        return f'unaligned_path : {self.unaligned_path} \naligned_path : {self.aligned_path} \nquery : {self.query} \nmyseq : {self.myseq}'

In [3]:
App1 = {'unaligned' : '../raw/App_all_seqs2.txt',
      'aligned' : '../processed/aln-App.txt',
      'query' : 'YTSI',
       'myseq' : 'sp|P05067|A4_HUMAN'}
App2 = {'unaligned' : '../raw/App_all_seqs2.txt',
      'aligned' : '../processed/aln-App.txt',
      'query' : 'YENPTY',
       'myseq' : 'sp|P05067|A4_HUMAN'}
CsgA = {'unaligned' : '../raw/CsgA_all_seqs2.txt',
      'aligned' : '../processed/aln-CsgA.txt',
      'query' : 'VVPQYG',
       'myseq' : 'sp|P28307|CSGA_ECOLI'}
Bap1 = {'unaligned' : '../raw/Bap_all_seqs2.txt',
      'aligned' : '../processed/aln-Bap.txt',
      'query' : 'DYDKDGLLDRYER',
       'myseq' : 'tr|Q79LN3|Q79LN3_STAAU'}
Bap2 = {'unaligned' : '../raw/Bap_all_seqs2.txt',
      'aligned' : '../processed/aln-Bap.txt',
      'query' : 'DTDGDGKNDGDEV',
       'myseq' : 'tr|Q79LN3|Q79LN3_STAAU'}

In [4]:
App1 = Protein(App1['unaligned'],App1['aligned'],App1['query'],App1['myseq'])

In [5]:
App2 = Protein(App2['unaligned'],App2['aligned'],App2['query'],App2['myseq'])

In [6]:
CsgA = Protein(CsgA['unaligned'],CsgA['aligned'],CsgA['query'],CsgA['myseq'])

In [7]:
Bap1 = Protein(Bap1['unaligned'],Bap1['aligned'],Bap1['query'],Bap1['myseq'])

In [8]:
Bap2 = Protein(Bap2['unaligned'],Bap2['aligned'],Bap2['query'],Bap2['myseq'])

In [9]:
prots = {
    'Bap1' : Bap1,
    'Bap2' : Bap2,
    'App1' : App1,
    'App2' : App2,
    'CsgA' : CsgA
}

## Loading motifs
Loading ELMs classes and candidate classes to use in the notebook

In [10]:
class_df = pd.read_csv('../raw/elm_classes.tsv', sep='\t')

In [11]:
cand_df = pd.read_html('http://elm.eu.org/elms/candidates')[1]

### Compiling RegExes
Compiling the patterns found in ELM to use in the notebook and discarding the miswritten ones.

In [12]:
cand_dict = {}
for element in cand_df.transpose():
    candidate = cand_df.transpose()[element]
    Regex = candidate['Model']
    ID = candidate['Identifier']
    try:
        print('reads wrong',ID,cand_dict[ID],Regex)
    except:
        cand_dict[ID] = Regex
new_cand_dict = {}
for ID in cand_dict:
    Regex = cand_dict[ID]
    try:
        pattern = re.compile(fr'{Regex}')
        new_cand_dict[ID] =  Regex
    except Exception as e:
        print(ID,Regex,e)
cand_dict = new_cand_dict

reads wrong LIG_VCP_VBM_2 [MILVAK][RK][^P][^P]R[LFWE][^P][^P][FLI][^P] nan
LIG_Menin_MBM_1 ...R{0,2)FP[GA].P unbalanced parenthesis at position 8
LIG_RCD1_1 [DE].{1,2)}[YF].{1,4}[DE]L unbalanced parenthesis at position 9
LIG_SUFU_Gli_1 [CY]GH[LF} unterminated character set at position 6


/tmp/ipykernel_1149434/2906291752.py:14: FutureWarning: Possible nested set at position 19
  pattern = re.compile(fr'{Regex}')


In [13]:
class_dict = {}
for element in class_df.transpose():
    candidate = class_df.transpose()[element]
    Regex = candidate['Regex']
    ID = candidate['ELMIdentifier']
    try:
        print('reads wrong',ID,class_dict[ID],Regex)
    except:
        class_dict[ID] = Regex

## Defining useful functions
we define functions to encapsulate behavior we want to use for each protein.
- get_aln_query_span(prot) takes a protein object and finds the positions in the alingment that have the regions of interest.
- get_classes(prot) takes a protein object and finds all the matches of the ELM classes in the interest region for each protein in the alignment and returns this as a dictionary of the form "{prot:{regex:{seq:[match1,match2...]}}}"
- get_candidates(prot) does the same as get_classes but uses the patterns found in the candidate classes of ELM.

In [14]:
def get_aln_query_span(prot):
    aln_pos_dict = {}
    query = prot.query
    myseqID = prot.myseq
    myseq_unaln = prot.unaligned_dict[myseqID]
    myseq_aln = prot.aligned_dict[myseqID]
    j = 0
    for i,letter in enumerate(myseq_aln):
        if letter != '-':
            aln_pos_dict[j] = i
            j += 1
    query_span = (aln_pos_dict[myseq_unaln.find(query)],aln_pos_dict[myseq_unaln.find(query) + len(query)])
    return query_span

In [15]:
#{prot:{regex:{seq:[match1,match2...]}}}
def get_classes(prot):
    regex_dict = {}
    myseqID = prot.myseq
    myseq_unaln = prot.unaligned_dict[myseqID]
    myseq_aln = prot.aligned_dict[myseqID]
    query_span = get_aln_query_span(prot)
    for classID in class_dict:
        Regex = class_dict[classID]
        pattern = re.compile(fr'{Regex}')
        sequence_dict = {}
        for seqID in prot.aligned_dict:
            seq = str(prot.aligned_dict[seqID])
            long_query = seq[query_span[0]-5:query_span[1]+5]
            short_query = seq[query_span[0]:query_span[1]]
            matches = pattern.finditer(long_query)
            true_matches = []
            for match in matches:
                match = long_query[match.span()[0]:match.span()[1]]
                if match.find(short_query) != -1:
                    true_matches.append(match)
            sequence_dict[seqID] = true_matches
        regex_dict[classID] = {'sequences' : sequence_dict,'pattern' : Regex}
    filtered_regex_dict = {}
    for regex in regex_dict:
        filtered_sequence_dict = {}
        sequence_dict = regex_dict[regex]['sequences']
        for seq in sequence_dict:
            matches = sequence_dict[seq]
            if len(matches) != 0:
                filtered_sequence_dict[seq] = matches
        if len(filtered_sequence_dict) != 0:
            filtered_regex_dict[regex] = filtered_sequence_dict
    return filtered_regex_dict

In [16]:
def get_candidates(prot):
    regex_dict = {}
    myseqID = prot.myseq
    myseq_unaln = prot.unaligned_dict[myseqID]
    myseq_aln = prot.aligned_dict[myseqID]
    query_span = get_aln_query_span(prot)
    for candID in cand_dict:
        Regex = cand_dict[candID]
        pattern = re.compile(fr'{Regex}')
        sequence_dict = {}
        for seqID in prot.aligned_dict:
            seq = str(prot.aligned_dict[seqID])
            long_query = seq[query_span[0]-5:query_span[1]+5]
            short_query = seq[query_span[0]:query_span[1]]
            matches = pattern.finditer(long_query)
            true_matches = []
            for match in matches:
                match = long_query[match.span()[0]:match.span()[1]]
                if match.find(short_query) != -1:
                    true_matches.append(match)
            sequence_dict[seqID] = true_matches
        regex_dict[candID] = {'sequences' : sequence_dict,'pattern' : Regex}
    filtered_regex_dict = {}
    for regex in regex_dict:
        filtered_sequence_dict = {}
        sequence_dict = regex_dict[regex]['sequences']
        for seq in sequence_dict:
            matches = sequence_dict[seq]
            if len(matches) != 0:
                filtered_sequence_dict[seq] = matches
        if len(filtered_sequence_dict) != 0:
            filtered_regex_dict[regex] = filtered_sequence_dict
    return filtered_regex_dict

## Getting matches
performing the search in all the interest regions of the proteins and save it as a dictionary indexed by the protein object name

In [17]:
full_dict = {}
regex_name_dict = {}
for name in prots:
    print(name)
    prot = prots[name]
    regex_name_dict[name] = {}
    classes_dict = get_classes(prot)
    for regex in classes_dict:
        try:
            regex_name_dict[name]['classes'].append(regex)
        except:
            regex_name_dict[name]['classes'] = [regex]
    candidates_dict = get_candidates(prot)
    for regex in candidates_dict:
        try:
            regex_name_dict[name]['candidates'].append(regex)
        except:
            regex_name_dict[name]['candidates'] = [regex]
    full_dict[name] = {'classes':classes_dict,'candidates':candidates_dict}

Bap1
Bap2


/tmp/ipykernel_1149434/876524001.py:9: FutureWarning: Possible nested set at position 19
  pattern = re.compile(fr'{Regex}')


App1
App2
CsgA


In [18]:
full_dict

{'Bap1': {'classes': {}, 'candidates': {}},
 'Bap2': {'classes': {}, 'candidates': {}},
 'App1': {'classes': {'DOC_MAPK_gen_1': {'XP_028835854.1': ['RKKQYMSI']},
   'DOC_WW_Pin1_4': {'XP_001601635.2': ['SARSPH'],
    'XP_046750444.1': ['SARSPH'],
    'XP_046432224.1': ['SARSPH'],
    'XP_011148932.1': ['SARSPH'],
    'XP_011639016.1': ['SARSPH'],
    'XP_011252362.1': ['SARSPH'],
    'XP_043288438.1': ['SARSPH'],
    'XP_044583405.1': ['SARSPH']},
   'LIG_14-3-3_CanoR_1': {'XP_001601635.2': ['RRSARSP'],
    'XP_046750444.1': ['RRSARSP'],
    'XP_046432224.1': ['RRSARSP'],
    'XP_011148932.1': ['RRSARSP'],
    'XP_011639016.1': ['RRSARSP'],
    'XP_011252362.1': ['RRSARSP'],
    'XP_043288438.1': ['RRSARSP'],
    'XP_044583405.1': ['RRSARSP']},
   'LIG_SH2_CRK': {'sp|P05067|A4_HUMAN': ['YTSIH'],
    'XP_028835854.1': ['YMSIH'],
    'XP_043875633.1': ['YTSIH'],
    'XP_034431105.1': ['YTSIH'],
    'XP_030613788.1': ['YTSIH'],
    'XP_022051540.2': ['YTSIH'],
    'XP_018527544.1': ['YTSI

## Dissecting the results
We show the results for each protein and we show the main classes obtained in the process

In [21]:
full_dict["Bap1"]

{'classes': {}, 'candidates': {}}

In [22]:
full_dict["Bap2"]

{'classes': {}, 'candidates': {}}

In [23]:
full_dict["App1"]

{'classes': {'DOC_MAPK_gen_1': {'XP_028835854.1': ['RKKQYMSI']},
  'DOC_WW_Pin1_4': {'XP_001601635.2': ['SARSPH'],
   'XP_046750444.1': ['SARSPH'],
   'XP_046432224.1': ['SARSPH'],
   'XP_011148932.1': ['SARSPH'],
   'XP_011639016.1': ['SARSPH'],
   'XP_011252362.1': ['SARSPH'],
   'XP_043288438.1': ['SARSPH'],
   'XP_044583405.1': ['SARSPH']},
  'LIG_14-3-3_CanoR_1': {'XP_001601635.2': ['RRSARSP'],
   'XP_046750444.1': ['RRSARSP'],
   'XP_046432224.1': ['RRSARSP'],
   'XP_011148932.1': ['RRSARSP'],
   'XP_011639016.1': ['RRSARSP'],
   'XP_011252362.1': ['RRSARSP'],
   'XP_043288438.1': ['RRSARSP'],
   'XP_044583405.1': ['RRSARSP']},
  'LIG_SH2_CRK': {'sp|P05067|A4_HUMAN': ['YTSIH'],
   'XP_028835854.1': ['YMSIH'],
   'XP_043875633.1': ['YTSIH'],
   'XP_034431105.1': ['YTSIH'],
   'XP_030613788.1': ['YTSIH'],
   'XP_022051540.2': ['YTSIH'],
   'XP_018527544.1': ['YTSIH'],
   'XP_018527459.1': ['YTSIH'],
   'XP_042282586.1': ['YTSIH'],
   'XP_044072518.1': ['YTSIH'],
   'XP_033833649.1'

In [25]:
[(name, matches[App1.myseq]) for name, matches in full_dict["App1"]["classes"].items() if App1.myseq in matches]

[('LIG_SH2_CRK', ['YTSIH']),
 ('LIG_SH2_STAT5', ['YTSI']),
 ('TRG_ENDOCYTIC_2', ['YTSI'])]

In [27]:
full_dict["App2"]

{'classes': {'LIG_PTB_Apo_2': {'sp|P05067|A4_HUMAN': ['GYENPTYK'],
   'XP_028835854.1': ['GYENPTYK'],
   'XP_043875633.1': ['GYENPTYK'],
   'XP_034431105.1': ['GYENPTYK'],
   'XP_030613788.1': ['GYENPTYK'],
   'XP_022051540.2': ['GYENPTYK'],
   'XP_018527544.1': ['GYENPTYK'],
   'XP_018527459.1': ['GYENPTYK'],
   'XP_042282586.1': ['GYENPTYK'],
   'XP_044072518.1': ['GYENPTYK'],
   'XP_033833649.1': ['GYENPTYK'],
   'XP_029030763.1': ['GYENPTYK'],
   'XP_030600646.1': ['GYENPTYK'],
   'XP_030600638.1': ['GYENPTYK'],
   'XP_029941707.1': ['GYENPTYK'],
   'XP_044043247.1': ['GYENPTYK'],
   'XP_038574583.1': ['GYENPTYK'],
   'XP_031438672.1': ['GYENPTYK'],
   'XP_041935309.1': ['GYENPTYK'],
   'XP_043104465.1': ['GYENPTYK'],
   'XP_036445455.1': ['GYENPTYK'],
   'XP_016096102.1': ['GYENPTYK'],
   'XP_026861393.2': ['GYENPTYK'],
   'XP_020360042.1': ['GYENPTYK'],
   'XP_046178998.1': ['GYENPTYK'],
   'XP_029596311.1': ['GYENPTYK'],
   'XP_035610824.1': ['GYENPTYK'],
   'XP_021464640.2': ['

In [28]:
[(name, matches[App2.myseq]) for name, matches in full_dict["App2"]["classes"].items() if App2.myseq in matches]

[('LIG_PTB_Apo_2', ['GYENPTYK']), ('LIG_PTB_Phospho_1', ['GYENPTY'])]

In [29]:
full_dict["CsgA"]

{'classes': {}, 'candidates': {}}

# Results
We got matches only in App, the classes we found for the YENPTY and YTSI regiones were:
**LIG_SH2_CRK**, **LIG_SH2_STAT5** and **TRG_ENDOCYTIC_2** for YTSI and **LIG_PTB_Apo_2** and **LIG_PTB_Phospho_1** for YENPTY, we then found that the **LIG_PTB_Apo_2** match is actually an anotated instance of App in that position, whereas none of the matches found on the YTSI motif were.